# Ranking Images by Visual Suspicion

When working with large image collections, the goal is often not to make
a final decision on every image, but to decide where human attention
should be focused first.

VAAS is well suited for this task because it produces a scalar anomaly
score (S_H) that can be used to rank images by visual inconsistency.

This notebook demonstrates how to:
- analyze a collection of images,
- rank them by their VAAS anomaly score,
- and prioritize images for further inspection.


## Installation

Install VAAS if it is not already available in the environment.


In [ ]:
!pip install vaas==0.1.7

In [ ]:
from vaas.inference.pipeline import VAASPipeline
from PIL import Image
import requests
from io import BytesIO


## Loading the VAAS Pipeline

The pipeline is initialized once and reused across all images.
This ensures consistent scoring and avoids unnecessary overhead.


In [ ]:
pipeline = VAASPipeline.from_pretrained(
    "OBA-Research/vaas-v1-df2023",
    device="cpu",
    alpha=0.5,
)


## Defining an Image Collection

We define a small set of image URLs hosted in the VAAS GitHub repository.
This keeps the notebook easy to run while illustrating the ranking process.

In real applications, this list may contain hundreds or thousands of images.


In [ ]:
image_urls = [
    "https://raw.githubusercontent.com/OBA-Research/VAAS/main/examples/images/COCO_DF_C110B00000_00539519.jpg",
    "https://raw.githubusercontent.com/OBA-Research/VAAS/main/examples/images/COCO_DF_E000B00011_00834527.jpg",
    "https://raw.githubusercontent.com/OBA-Research/VAAS/main/examples/images/COCO_DF_E000B04010_00863031.jpg",
]


In [ ]:
def load_image_from_url(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert("RGB")

## Computing Anomaly Scores

We iterate over the image collection and compute VAAS scores for each image.
Only the hybrid score (S_H) is used for ranking.


In [ ]:
results = []

for url in image_urls:
    image = load_image_from_url(url)
    output = pipeline(image)

    results.append(
        {
            "image_url": url,
            "S_H": output["S_H"],
        }
    )


In [ ]:
ranked_results = sorted(results, key=lambda x: x["S_H"], reverse=True)

## Ranked Results

The images below are ordered from most suspicious to least suspicious
according to VAAS.


In [ ]:
for idx, r in enumerate(ranked_results, start=1):
    print(f"Rank {idx}")
    print(f"  Image URL: {r['image_url']}")
    print(f"  Hybrid score (S_H): {r['S_H']:.4f}")
    print()

## Visual Inspection of Top-Ranked Images

After ranking, it is common to visualize only the top few images.
This keeps analysis efficient while maintaining interpretability.


In [ ]:
from IPython.display import Image as IPImage, display

top_image_url = ranked_results[0]["image_url"]
top_image = load_image_from_url(top_image_url)

output_path = "vaas_top_ranked_visualization.png"

pipeline.visualize(
    image=top_image,
    save_path=output_path,
    mode="all",
    threshold=0.5,
)

display(IPImage(filename=output_path))



## Practical Use Cases

Ranking by VAAS anomaly score is useful for:
- forensic triage of large media collections,
- dataset cleaning and quality control,
- prioritizing human review in investigative workflows.

VAAS is most effective when used as a decision-support tool
rather than an automated verdict system.


## Limitations

With the current VAAS API:
- ranking logic is implemented externally,
- no dataset abstraction is provided,
- scores should not be interpreted as probabilities.

These limitations keep the system transparent and adaptable.


## Summary

In this notebook, we:
- computed anomaly scores for a collection of images,
- ranked images by visual inconsistency,
- and demonstrated selective visualization for triage.

The next notebook will focus on how researchers can reuse VAAS outputs
in downstream analysis and experimentation.

check it out: [09_using_vaas_outputs_in_downstream_research.ipynb](https://colab.research.google.com/drive/1AiciR4GcXimFgr7M8Q8fXFCTekpmXN_X?usp=sharing)
